<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=d83f448af7395d94cb3dcfbcbe6d335a62c46f2f22983525295d933d6d7e626e
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-29 10:41:42
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: 59.37 K (0.41%)
Current PnL: -18.70 L (-12.25%)
CY Booked + Current PnL: -6.57 L (-4.3%)
-------------------
Total profit:  2.68 L
Total loss:  -21.37 L
-------------------
Total Booked + Current PnL: 20.41 L (16.25%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.34%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.32 C
Est FTT PnL: 86.40 L (59.38%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.76%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-6.16,76.0,X-LC,4.17,234468.0,24102.0,4689.0,1.08,11.46,2.00,13.68,37.0,5.14,1.63,29.87,XY25,NTT,REFINERIES
79,TTKPRESTIG,770.0,111.95,66.0,M-SC,1.42,91340.0,-9437.0,9527.0,-2.64,-9.36,10.43,0.09,245.0,-0.99,0.63,20.00,OX40N,NTT,DURABLES
42,ITC,452.0,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
85,VOLTAS,1530.0,1.58,54.0,X-MC,3.91,213000.0,21258.0,16508.0,-0.59,11.09,7.75,19.69,99.0,1.29,1.48,18.64,XY25,NTT,AC
2,ABBOTINDIA,35195.0,-9.84,40.0,X-MC,2.76,88995.0,-1683.0,16589.0,-0.35,-1.86,18.64,16.44,101.0,-0.10,0.62,17.95,X40,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,55.57,69.0,M-MC,10.21,245488.0,20526.0,147121.0,7.87,9.12,59.93,74.53,192.0,0.14,1.70,44.32,XY24,BTT,STEEL
82,VAIBHAVGBL,521.00,65.29,69.0,H-SC,3.83,143250.0,-39525.0,150599.0,5.86,-21.63,105.13,60.77,125.0,-0.26,0.99,28.30,XR,NTT,JEWELLERY
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211127.0,6051.0,106514.0,3.00,2.95,50.45,54.89,52.0,0.06,1.47,27.76,X5K,ATH,METALS
84,VBL,671.64,-17.18,57.0,X-LC,9.18,298106.0,-17736.0,130392.0,2.88,-5.62,43.74,35.67,5.0,-0.14,2.07,7.46,X40N,ATH,FMCG
66,SHALBY,327.00,1158.37,52.0,M-SC,21.27,171102.0,-10041.0,55505.0,2.41,-5.54,32.44,25.10,235.0,-0.18,1.19,37.12,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.00,13.67,45.0,H-SC,12.45,252987.0,-28136.0,161001.0,-5.98,-10.01,63.64,47.26,144.0,-0.17,1.76,35.54,XY24,NTT,INSURANCE
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.56,91383.0,-37287.0,82583.0,-5.59,-28.98,90.37,35.20,219.0,-0.45,0.63,4.89,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-5.16,49.0,X-SC,0.42,107940.0,5936.0,37984.0,-3.27,5.82,35.19,43.06,122.0,0.16,0.75,23.93,X40N,ATH,MISC
5,ANGELONE,3033.00,14.29,58.0,X-SC,5.46,202466.0,11460.0,46243.0,-2.71,6.00,22.84,30.21,157.0,0.25,1.41,28.13,X40N,NTT,FINANCE
79,TTKPRESTIG,770.00,111.95,66.0,M-SC,1.42,91340.0,-9437.0,9527.0,-2.64,-9.36,10.43,0.09,245.0,-0.99,0.63,20.00,OX40N,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.0,-29.46,65.0,H-SC,16.15,201704.0,1750.0,95487.0,0.76,0.88,47.34,48.63,115.0,0.02,1.40,12.53,XR,NTT,MISC
87,WIPRO,420.0,-14.25,46.0,M-LC,5.71,150610.0,-335.0,109795.0,0.22,-0.22,72.90,72.51,53.0,-0.00,1.05,5.79,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,111.95,66.0,M-SC,1.42,91340.0,-9437.0,9527.0,-2.64,-9.36,10.43,0.09,245.0,-0.99,0.63,20.00,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.24,53.0,H-SC,2.73,226026.0,-43641.0,79968.0,0.66,-16.18,35.38,13.47,138.0,-0.55,1.57,14.93,XY24,NTT,PAINTS
74,TATAELXSI,9161.0,-22.37,53.0,H-MC,6.77,104928.0,-26907.0,69137.0,-0.41,-20.41,65.89,32.03,98.0,-0.39,0.73,17.53,OX40N,NTT,IT
68,SIS,528.0,2041.97,47.0,H-SC,3.15,86445.0,-24587.0,48193.0,-0.86,-22.14,55.75,21.26,156.0,-0.51,0.60,16.80,OX40N,NTT,MISC
18,CERA,9475.0,-24.82,38.0,H-SC,2.67,136930.0,-38973.0,80994.0,-0.23,-22.16,59.15,23.89,149.0,-0.48,0.95,18.26,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.0,-15.53,75.0,H-MC,16.24,216750.0,36942.0,75472.0,-0.11,20.55,34.82,62.52,88.0,0.49,1.5,61.64,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,55.57,69.0,M-MC,10.21,245488.0,20526.0,147121.0,7.87,9.12,59.93,74.53,192.0,0.14,1.70,44.32,XY24,BTT,STEEL
57,RECLTD,446.00,45.36,54.0,M-LC,5.51,206030.0,2860.0,39269.0,1.53,1.41,19.06,20.74,55.0,0.07,1.43,8.37,XY25,NTT,FINANCE
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211127.0,6051.0,106514.0,3.00,2.95,50.45,54.89,52.0,0.06,1.47,27.76,X5K,ATH,METALS
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.36,131342.0,6045.0,52878.0,-0.28,4.82,40.26,47.03,141.0,0.11,0.91,64.45,MH,ATH,POWER
80,UNIONBANK,163.00,-8.53,65.0,M-LC,9.42,166525.0,25685.0,18801.0,-0.41,18.24,11.29,31.59,66.0,1.37,1.16,49.56,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.24,216750.0,36942.0,75472.0,-0.11,20.55,34.82,62.52,88.0,0.49,1.50,61.64,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-38.27,64.0,M-SC,3.36,127871.0,13073.0,80968.0,0.63,11.39,63.32,81.92,223.0,0.16,0.89,51.38,XR,NTT,DURABLES
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.36,131342.0,6045.0,52878.0,-0.28,4.82,40.26,47.03,141.0,0.11,0.91,64.45,MH,ATH,POWER
32,HINDZINC,730.22,29.80,52.0,M-LC,11.39,211127.0,6051.0,106514.0,3.00,2.95,50.45,54.89,52.0,0.06,1.47,27.76,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.71,53.0,H-SC,6.77,129400.0,6064.0,125570.0,-1.57,4.92,97.04,106.72,119.0,0.05,0.90,29.44,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,19.0,X-LC,18.69,149374.0,-127149.0,237221.0,0.35,-45.98,158.81,39.81,54.0,-0.54,1.04,5.28,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.56,91383.0,-37287.0,82583.0,-5.59,-28.98,90.37,35.20,219.0,-0.45,0.63,4.89,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-30.68,37.0,X-MC,2.56,200730.0,-90.0,67887.0,-0.12,-0.04,33.82,33.76,80.0,-0.00,1.39,5.15,X40,ATH,FMCG
24,DMART,5391.45,-16.90,38.0,X-LC,9.89,101630.0,-956.0,27765.0,0.32,-0.93,27.32,26.13,38.0,-0.03,0.71,24.40,X40N,ATH,FMCG
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.76,88995.0,-1683.0,16589.0,-0.35,-1.86,18.64,16.44,101.0,-0.10,0.62,17.95,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.50,-1.97,50.0,H-LC,1.06,157615.0,-28480.0,75955.0,1.37,-15.30,48.19,25.51,15.0,-0.37,1.09,16.15,AR,ATH,ELECTRICAL
42,ITC,452.00,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
53,PGHH,17907.65,-30.68,37.0,X-MC,2.56,200730.0,-90.0,67887.0,-0.12,-0.04,33.82,33.76,80.0,-0.00,1.39,5.15,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.47,50.0,X-MC,3.03,315462.0,-12529.0,119055.0,1.46,-3.82,37.74,32.48,92.0,-0.11,2.19,7.17,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.58,54.0,X-MC,3.91,213000.0,21258.0,16508.0,-0.59,11.09,7.75,19.69,99.0,1.29,1.48,18.64,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
10,BAJAJHFL,181.50,-14.48,46.0,X-MC,5.94,180170.0,-20336.0,117489.0,0.78,-10.14,65.21,48.45,90.0,-0.17,1.25,3.90,X40N,ATH,FINANCE
52,PAGEIND,51769.93,-27.30,41.0,X-MC,6.80,82180.0,-1330.0,21359.0,0.43,-1.59,25.99,23.98,82.0,-0.06,0.57,4.13,X40,ATH,APPARELS
21,COLPAL,3767.14,-3.66,49.0,X-MC,7.87,225060.0,-38305.0,151645.0,0.80,-14.54,67.38,43.04,84.0,-0.25,1.56,4.48,XY25,ATH,FMCG
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.56,91383.0,-37287.0,82583.0,-5.59,-28.98,90.37,35.20,219.0,-0.45,0.63,4.89,X40,NTT,FOOTWEAR
3,ACC,3906.00,-36.77,57.0,X-MC,2.54,187420.0,-50531.0,203182.0,0.73,-21.24,108.41,64.15,174.0,-0.25,1.30,5.13,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-45.45,50.0,X-SC,38.22,54060.0,-10946.0,161872.0,0.24,-16.84,299.43,232.18,198.0,-0.07,0.38,5.45,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.30,41.0,X-MC,6.80,82180.0,-1330.0,21359.0,0.43,-1.59,25.99,23.98,82.0,-0.06,0.57,4.13,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.30,46.0,X-SC,4.81,82748.0,-62412.0,138338.0,1.31,-43.00,167.18,52.31,136.0,-0.45,0.57,10.76,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-9.84,40.0,X-MC,2.76,88995.0,-1683.0,16589.0,-0.35,-1.86,18.64,16.44,101.0,-0.10,0.62,17.95,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-38.87,32.0,X-SC,4.56,91383.0,-37287.0,82583.0,-5.59,-28.98,90.37,35.20,219.0,-0.45,0.63,4.89,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,56.0,X-LC,11.29,288477.0,-57479.0,124189.0,0.36,-16.61,43.05,19.28,1.0,-0.46,2.00,6.65,X40,ATH,IT
40,INFY,2275.00,-17.80,58.0,X-LC,6.56,319543.0,6655.0,165044.0,-0.29,2.13,51.65,54.87,3.0,0.04,2.22,10.59,X40,BTT,IT
42,ITC,452.00,-36.66,64.0,X-LC,1.41,204842.0,4704.0,16183.0,0.24,2.35,7.90,10.44,4.0,0.29,1.42,8.04,X40,NTT,FMCG
84,VBL,671.64,-17.18,57.0,X-LC,9.18,298106.0,-17736.0,130392.0,2.88,-5.62,43.74,35.67,5.0,-0.14,2.07,7.46,X40N,ATH,FMCG
1,ABB,7934.00,-39.38,56.0,H-LC,5.80,253320.0,-8299.0,127521.0,1.97,-3.17,50.34,45.57,7.0,-0.07,1.76,7.79,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7133.33,64.0,L-SC,7.62,79367.0,-14443.0,91058.0,0.06,-15.40,114.73,81.67,269.0,-0.16,0.55,53.70,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,27.93,81485.0,-32064.0,72114.0,-0.35,-28.24,88.50,35.27,268.0,-0.44,0.57,99.44,XR,NTT,HOTELS
50,MASFIN,398.61,-20.51,45.0,H-SC,9.14,90570.0,-7410.0,29010.0,0.07,-7.56,32.03,22.05,152.0,-0.26,0.63,31.65,XR,ATH,FINANCE
79,TTKPRESTIG,770.00,111.95,66.0,M-SC,1.42,91340.0,-9437.0,9527.0,-2.64,-9.36,10.43,0.09,245.0,-0.99,0.63,20.00,OX40N,NTT,DURABLES
86,WHIRLPOOL,2270.00,-38.27,64.0,M-SC,3.36,127871.0,13073.0,80968.0,0.63,11.39,63.32,81.92,223.0,0.16,0.89,51.38,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SONACOMS,806.63,-31.80,72.0,M-SC,7.97,84840.0,-16319.0,56317.0,0.09,-16.13,66.38,39.54,202.0,-0.29,0.59,19.63,AR,ATH,AUTO
63,SAMMAANCAP,326.00,-195.45,72.0,M-SC,29.60,164790.0,15570.0,128602.0,-2.22,10.43,78.04,96.62,208.0,0.12,1.14,78.34,XY25,NTT,FINANCE
12,BANKINDIA,190.00,-15.53,75.0,H-MC,16.24,216750.0,36942.0,75472.0,-0.11,20.55,34.82,62.52,88.0,0.49,1.50,61.64,XR,NTT,BANKS
47,KPIGREEN,731.05,10.17,71.0,H-SC,5.36,131342.0,6045.0,52878.0,-0.28,4.82,40.26,47.03,141.0,0.11,0.91,64.45,MH,ATH,POWER
5,ANGELONE,3033.00,14.29,58.0,X-SC,5.46,202466.0,11460.0,46243.0,-2.71,6.00,22.84,30.21,157.0,0.25,1.41,28.13,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,43.96
2,50,74.91


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.94
LC    32.36
MC    24.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.09
X40      16.26
XY25     12.06
XR       11.45
X40N     10.58
AR        7.83
OX40N     6.51
X200      1.76
X5K       1.47
SR        1.07
MH        0.91
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.31
X-LC    21.62
X-MC    18.78
M-SC    12.67
M-LC     5.11
H-LC     4.61
X-SC     4.51
H-MC     3.89
M-MC     1.70
L-SC     1.45
L-LC     1.02
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.46,-3.43,37.18
IT,12.49,-16.80,79.44
FINANCE,12.03,-9.08,58.53
MISC,6.86,-16.54,81.11
ELECTRICAL,5.81,-7.87,46.11
BANKS,5.67,-4.73,66.61
PAINTS,5.50,-16.50,33.86
INSURANCE,3.75,-2.66,40.94
AUTO,2.77,-44.15,104.30


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3242011.0,22
XR,1298609.0,14
AR,1161562.0,9
X40,814034.0,12
X40N,636957.0,9
OX40N,546218.0,9
XY25,444213.0,8
SR,266386.0,2
X5K,106514.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3327763.0,24
M-SC,1220366.0,15
X-MC,1172925.0,14
X-LC,1026213.0,13
X-SC,530743.0,6
H-MC,319920.0,3
M-LC,274379.0,4
H-LC,273817.0,3
L-SC,252568.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1163976.0      6
           AR         834537.0      5
           XR         765798.0      7
M-SC       XY24       697262.0      6
X-MC       XY24       573501.0      4
X-LC       X40        447026.0      5
           XY24       304167.0      2
X-SC       X40N       286288.0      4
X-MC       X40        284425.0      6
H-SC       SR         266386.0      2
           OX40N      244188.0      3
X-LC       X40N       203823.0      3
H-LC       AR         203476.0      2
H-MC       XY24       175311.0      1
X-MC       XY25       168153.0      2
L-SC       XR         163172.0      2
X-SC       XY24       161872.0      1
M-MC       XY24       147121.0      1
X-MC       X40N       146846.0      2
M-SC       OX40N      143497.0      4
           XY25       128602.0      1
           XR         127456.0      2
           AR         123549.0      2
M-LC       XR         109795.0      1
           X5K        106514.0      1
L-SC       OX40N       89396.0      1
X-SC       X40         82583.0      1
H-MC       XR          75472.0      1
X-LC       XY25        71197.0      3
H-LC       X200        70341.0      1
H-MC       OX40N       69137.0      1
L-MC       XR          56916.0      1
H-SC       MH          52878.0      1
M-LC       XY25        39269.0      1
L-LC       XY25        36992.0      1
M-LC       XY24        18801.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
